In [26]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
!pip install numpy==1.26.4


In [1]:
import numpy
print(numpy.__version__)


1.26.4


In [44]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import torch

from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api

In [10]:
word_vectors = api.load('glove-wiki-gigaword-100')

[==================================================] 100.0% 128.1/128.1MB downloaded


In [28]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [11]:
# Function to get BERT embeddings
def get_bert_embeddings(sentence, word):
    inputs = tokenizer(sentence, return_tensors='pt')
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    word_tokens = tokenizer.tokenize(sentence)
    word_index = word_tokens.index(word)
    word_embedding = last_hidden_states[0, word_index + 1, :]  # +1 to account for [CLS] token
    return word_embedding

In [12]:
sentence1 = "The bat flew out of the cave at night."
sentence2 = "He swung the bat and hit a home run."

word = "bat"

bert_embedding1 = get_bert_embeddings(sentence1, word).detach().numpy()
bert_embedding2 = get_bert_embeddings(sentence2, word).detach().numpy()
word_embedding = word_vectors[word]

In [47]:
word_embd = api.load('glove-wiki-gigaword-100')

In [52]:
word = "world"
embedding = word_embd[word]
embedding[:10]

array([ 0.49177  ,  1.1164   ,  1.1424   ,  0.14381  , -0.10696  ,
       -0.46727  , -0.44374  , -0.0088024, -0.50406  , -0.20549  ],
      dtype=float32)

In [17]:
cosine_similarity([bert_embedding1], [bert_embedding2])[0][0]

0.45995745

In [ ]:
!pip install sentence_transformers

In [31]:
from sentence_transformers import CrossEncoder
CrossEncoder_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', max_length=512, 
                     default_activation_function=torch.nn.Sigmoid())

In [41]:
question = "Where is the capital of France?"
# Define sentences to compare
answers = [
    "Paris is the capital of the France.",
    "Berlin is the capital of Germany.",
    "Madrid is the capital of Spain.",
    "Where is the capital of France?"
]

In [42]:
scores = CrossEncoder_model.predict([(question, answers[0]), (question, answers[1]),(question, answers[2]), (question,answers[3])])
print(scores)
most_relevant_idx = torch.argmax(torch.tensor(scores)).item()
print(most_relevant_idx)
print(f"The most relevant passage is: {answers[most_relevant_idx]}")

[0.9996613  0.05289618 0.04520683 0.99539846]
0
The most relevant passage is: Paris is the capital of the France.
